In [ ]:
import super_gradients as sg

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torchvision.transforms as T
import albumentations as A
import numpy as np
import torch

from dataset import YOLODataset, detection_collate_fn

In [ ]:
transforms = A.Compose(
    [
        A.ToFloat(max_value=255),
        A.HorizontalFlip(),
        A.Resize(320, 320),
    ],
    bbox_params=A.BboxParams(format='yolo', label_fields=['labels', 'indexes'])
    )

In [ ]:
transforms = A.Compose(
    [
        A.ToFloat(max_value=255),
        A.HorizontalFlip(),
        A.Resize(320, 320),
    ],
    bbox_params=A.BboxParams(format='yolo', label_fields=['labels', 'indexes'])
    )

train_dataset = YOLODataset(images_dir='/datasets/hituav-a-highaltitude-infrared-thermal-dataset/hit-uav/images/train',
                      labels_dir='/datasets/hituav-a-highaltitude-infrared-thermal-dataset/hit-uav/labels/train',
                      transforms=transforms,
                      class_names=['Person', 'Car', 'Bicycle', 'OtherVehicle', 'DontCare'])

train_dataset.plot()

In [ ]:
transforms = A.Compose(
    [
        A.ToFloat(max_value=255),
        A.Resize(320, 320),
    ],
    bbox_params=A.BboxParams(format='yolo', label_fields=['labels', 'indexes'])
    )

valid_dataset = YOLODataset(images_dir='/datasets/hituav-a-highaltitude-infrared-thermal-dataset/hit-uav/images/val',
                      labels_dir='/datasets/hituav-a-highaltitude-infrared-thermal-dataset/hit-uav/labels/val',
                      transforms=transforms,
                      class_names=['Person', 'Car', 'Bicycle', 'OtherVehicle', 'DontCare'])

valid_dataset.plot()

In [ ]:
def detection_collate_fn(raw_batch):

    images = []      # [B, H, W, C] or [B, C, H, W]
    annotations = [] # [B, ND*6], ND: #detections

    for i, _ in enumerate(raw_batch):
        image = raw_batch[i][0]
        annotation =  raw_batch[i][1]

        annotation = np.array(annotation).reshape(-1,6)

        images.append(image)
        annotations.append( np.array(annotation).reshape(-1,6) )

    batch_images = torch.from_numpy(np.array( images ))
    batch_annotations = torch.from_numpy( np.vstack( annotations )  )

    return batch_images, batch_annotations

In [ ]:
# annotations = [np.random.uniform(0, 1, size=[8*nd, 6]) for nd in range(1, 10)]
# np.vstack(annotations).shape

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=detection_collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False, collate_fn=detection_collate_fn)

In [ ]:
# Global parameters
EPOCHS = 10
BATCH_SIZE = 8
WORKERS = 4
IMAGE_SIZE = 320

ROOT_DIR = '/datasets/hit-uav'
train_imgs_dir = 'images/train'
train_labels_dir = 'labels/train'

val_imgs_dir = 'images/val'
val_labels_dir = 'labels/val'

test_imgs_dir = 'images/test'
test_labels_dir = 'labels/test'

classes = ['Person', 'Car', 'Bicycle', 'OtherVechicle', 'DontCare']

In [ ]:
from super_gradients.training import Trainer
from super_gradients.training import dataloaders
from super_gradients.training import transforms
from super_gradients.training.datasets import data_augmentation
from super_gradients.training import models
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import (
    DetectionMetrics_050,
    DetectionMetrics_050_095
)
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback
from tqdm.auto import tqdm

import matplotlib.pyplot as plt

In [ ]:
train_params = {
    'silent_mode': False,
    'sync_bn': True,
    'average_best_models': True,
    'warmup_mode':'linear_epoch_step',
    'warmup_initial_lr': 1e-6,
    'lr_warmup_epochs':3,
    'initial_lr':2e-3,
    'lr_mode':'cosine',
    'cosine_final_lr_ratio':0.1,
    'optimizer':'AdamW',
    'optimizer_params':{'weight_decay':0.0001},
    'zero_weight_decay_on_bias_and_bn': False,
    'ema': True,
    'ema_params': {'decay':0.9997, 'decay_type': 'threshold'},
    'max_epochs': EPOCHS,
    'mixed_precision':False,
    'loss': PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(classes),
        reg_max=16
    ),
    'valid_metrics_list':[
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(classes),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7,
            )
        ),
        DetectionMetrics_050_095(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(classes),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7,
            )
        )
    ],
    'metric_to_watch':'mAP@0.50:0.95',
    'greater_metric_to_watch_is_better': True,
}

In [ ]:
# ppyoloe_s, ppyoloe_m, ppyoloe_l, ppyoloe_x
model = sg.training.models.get(model_name=sg.training.models.Models.PP_YOLOE_S, pretrained_weights='coco', num_classes=len(classes))

In [ ]:
trainer = sg.Trainer(experiment_name='test-custom-dataloader', ckpt_root_dir='/workspace/ckps')

trainer.train(model=model,
              train_loader = train_loader,
              valid_loader = valid_loader,
              training_params=train_params)

In [ ]:
ib, ab = next(iter(valid_loader))

In [ ]:
ib.shape

In [ ]:
ib

In [ ]:
model.to('cpu')
model.eval()
out = model.forward( ib )

In [ ]:
out[0][1].shape

In [ ]:
PPYoloEPostPredictionCallback(0.1, 0.1, 10, 1000)(out, 'cpu')

# MyDataset & Dataloader

In [ ]:
# 0: Person
# 1: Car
# 2: Bicycle
# 3: OtherVehicle
# 4: DontCare
yolodataset = sg.training.datasets.YoloDarknetFormatDetectionDataset(data_dir='/datasets/hituav-a-highaltitude-infrared-thermal-dataset/hit-uav',
                                                       images_dir='images/test',
                                                       labels_dir='labels/test',
                                                       classes=['Person', 'Car', 'OtherVehicle', 'DontCare'])

In [ ]:
ib, lb = next(iter(yolodataset))

In [ ]:
yolodataset = sg.training.datasets.YoloDarknetFormatDetectionDataset(data_dir='/datasets/hituav-a-highaltitude-infrared-thermal-dataset/hit-uav',
                                                        images_dir='images/test',
                                                        labels_dir='labels/test',
                                                        classes=['Person', 'Car', 'OtherVehicle', 'DontCare'])

In [ ]:
ib, lb = next(iter(yolodataset))

In [ ]:
lb.shape

In [ ]:
yolodataset_loader = DataLoader(yolodataset, batch_size=8)

In [ ]:
ib, lb = next(iter(yolodataset_loader))

In [ ]:
# yolodataset = sg.training.datasets.YoloDarknetFormatDetectionDataset(data_dir='/datasets/hituav-a-highaltitude-infrared-thermal-dataset/hit-uav',
#                                                        images_dir='images/test',
#                                                        labels_dir='labels/test',
#                                                        classes=['Person', 'Car', 'OtherVehicle', 'DontCare'])

dataset = YOLODataset(images_dir='/datasets/hituav-a-highaltitude-infrared-thermal-dataset/hit-uav/images/test',
                      labels_dir='/datasets/hituav-a-highaltitude-infrared-thermal-dataset/hit-uav/labels/test',
                      transforms=None,
                      class_names=['Person', 'Car', 'OtherVehicle', 'DontCare'])

In [ ]:
image, annotations = dataset.__getitem__(0)
print(image.shape)
print(annotations.shape)

In [ ]:
len(dataset)

In [ ]:
dataset.plot()